# Debug notebook
This notebook is only used to troubleshoot the Landsat ARD workflow processor.

## Landsat ARD workflow

In [1]:
import json
from utils.prepLS import prepareLS

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    print("%s: %s" % (x, loaded_json[x]))
    prepareLS(**loaded_json)

no gsutils installed
sentinelsat not installed


## Landsat ARD job queue processor

In [2]:
import rediswq

import os
host = os.getenv("REDIS_SERVICE_HOST", "redis-master")

q = rediswq.RedisWQ(name="jobLS", host=host)
print("Worker with sessionID: " +  q.sessionID())
print("Initial queue state: empty=" + str(q.empty()))

Worker with sessionID: 032e251c-a1a6-45cb-a80e-e88d9f53bad9
Initial queue state: empty=False


In [ ]:
while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        print("Working on " + itemstr)
        #time.sleep(10) # Put your actual work here instead of sleep.
        process_scene(itemstr)
        q.complete(item)
    else:
        print("Waiting for work")

Working on {"in_scene": "https://edclpdsftp.cr.usgs.gov/orders/espa-Sarah.Cheesbrough@sa.catapult.org.uk-10252019-060110-664/LC080740722019072401T1-SC20191025121311.tar.gz", "s3_bucket": "public-eo-data", "s3_dir": "common_sensing/fiji/landsat_8_luigis_testing/"}
scene: LC08_L1TP_074072_20190724
untar: /tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC08_L1TP_074072_20190724_untar/
cog_dir/tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC08_L1TP_074072_20190724/
Extracting tar.gz: /tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC080740722019072401T1-SC20191025121311.tar.gz
/tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC08_L1TP_074072_20190724_untar/LC08_L1TP_074072_20190724_20190801_01_T1_sr_band7.tif
/tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC08_L1TP_074072_20190724_untar/LC08_L1TP_074072_20190724_20190801_01_T1_sr_band3.tif
/tmp/data/intermediate/LC08_L1TP_074072_20190724_tmp/LC08_L1TP_074072_20190724_untar/LC08_L1TP_074072_20190724_20190801_01_T1_

In [ ]:
print("Queue empty!")